In [16]:
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

import requests
from sentence_transformers import SentenceTransformer, util

with open('X_scaler_full_v1.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('xgboost_embedding_Lower_full_v1.pkl', 'rb') as file:
    best_model = pickle.load(file)

import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
revision = None  # Replace with the specific revision to ensure reproducibility if the model is updated.
model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0", revision=revision)

[nltk_data] Downloading package punkt to /home/eric/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eric/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/eric/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import numpy as np
import pandas as pd
from xgboost import DMatrix, train

In [18]:
bdpt_dict={}
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'

In [11]:
def create_df(X):

    sample_df_single_record=pd.DataFrame(X,columns=['Age', 'Sex','Location','Product_1' ,'activity_at_injury', 'object_involved','injury_mechanism','More_details'])

    sample_df_single_record["Age"] = pd.to_numeric(sample_df_single_record["Age"])
    sample_df_single_record["Sex"] = pd.to_numeric(sample_df_single_record["Sex"])
    sample_df_single_record["Location"] = pd.to_numeric(sample_df_single_record["Location"])
    sample_df_single_record["Product_1"] = pd.to_numeric(sample_df_single_record["Product_1"])

    sample_df_single_record['Narrative']=sample_df_single_record["activity_at_injury"].astype(str) + ' '+sample_df_single_record["injury_mechanism"].astype(str)+ ' ' + sample_df_single_record["object_involved"].astype(str)+' '+sample_df_single_record["More_details"].astype(str)

    Embedding_df=pd.DataFrame(model.encode(sample_df_single_record['Narrative'], convert_to_tensor=True).numpy())
    Embedding_df.columns=Embedding_df.columns.astype(str)

    sample_df_single_record_2=sample_df_single_record[["Age","Sex","Location","Product_1"]]

    return Embedding_df,sample_df_single_record_2

In [12]:
def gen_prediction(bdpt_dict,data_ready,scaler,best_model): #,sematic_distance_bert


    prob_out={}

    k=0

    for i in bdpt_dict:
        data_ready.at[0,'Body_Part']=i

        X_test_scaled = scaler.transform(data_ready.select_dtypes(include=['number']))
        X_test_final=pd.DataFrame(X_test_scaled, columns=data_ready.select_dtypes(include=['number']).columns)
        dtest = DMatrix(data=X_test_final)
        y_prob = best_model.predict(dtest)[0]
        #print(y_prob)
        prob_out[str(i)]=y_prob
        k+=1

    return prob_out

In [13]:
def Predict_Body_parts_Prob(Age, Sex,Location,Product_1 ,activity_at_injury, object_involved,injury_mechanism,More_details):

    X = np.column_stack([Age, Sex,Location,Product_1 ,activity_at_injury, object_involved,injury_mechanism,More_details])
    embedding_df,sample_df_single_record_2 = create_df(X)
    data = {"Body_Part": [25],}
    df_body= pd.DataFrame(data)
    data_ready = pd.concat([sample_df_single_record_2[['Age','Sex']],df_body,sample_df_single_record_2[['Location','Product_1']], embedding_df], axis=1)
    #print(data_ready)
    output = gen_prediction(bdpt_dict,data_ready,scaler,best_model)


    probabilities = [float(value) for value in output.values()]
    
    return probabilities

In [19]:
Predict_Body_parts_Prob(25, 1,1,420 ,"cutting paper", "","","")

[0.00026263290783390403,
 0.001730184187181294,
 0.0009127928642556071,
 0.001475128112360835,
 0.0006582694477401674,
 0.0007020221673883498,
 0.002235991647467017,
 0.0011580724967643619,
 0.00725253950804472,
 0.0013425503857433796,
 0.0007204681751318276,
 0.002093666000291705,
 0.0018697351915761828,
 0.0023155149538069963,
 0.005475759971886873,
 0.0009964368073269725,
 0.0013340895529836416,
 0.0008438182412646711,
 0.0018577511655166745,
 0.0005121431895531714,
 0.00044942618114873767,
 0.00026360625633969903]

In [15]:
from tabpy.tabpy_tools.client import Client

deploy = Client('http://tabpy.ericy.me:9004/')
deploy.set_credentials(username='dva023', password='YL8bar-_3.jXGFet')
deploy.deploy('tabpy_v6', Predict_Body_parts_Prob, 'tabpy v6 test', override=True)